In [31]:
import pandas as pd
import numpy as np

import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots

In [32]:
data_train = pd.read_csv('D:/Data/Tabular Playground July/train.csv')
data_test = pd.read_csv('D:/Data/Tabular Playground July/test.csv')

In [33]:
data_train

,date_time,deg_C,relative_humidity,absolute_humidity,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,target_carbon_monoxide,target_benzene,target_nitrogen_oxides
0,2010-03-10 18:00:00,13.1,46.0,0.7578,1387.2,1087.8,1056.0,1742.8,1293.4,2.5,12.0,167.7
1,2010-03-10 19:00:00,13.2,45.3,0.7255,1279.1,888.2,1197.5,1449.9,1010.9,2.1,9.9,98.9
2,2010-03-10 20:00:00,12.6,56.2,0.7502,1331.9,929.6,1060.2,1586.1,1117.0,2.2,9.2,127.1
3,2010-03-10 21:00:00,11.0,62.4,0.7867,1321.0,929.0,1102.9,1536.5,1263.2,2.2,9.7,177.2
4,2010-03-10 22:00:00,11.9,59.0,0.7888,1272.0,852.7,1180.9,1415.5,1132.2,1.5,6.4,121.8
...,...,...,...,...,...,...,...,...,...,...,...,...
7106,2010-12-31 20:00:00,9.2,32.0,0.3871,1000.5,811.2,873.0,909.0,910.5,1.3,5.1,191.1
7107,2010-12-31 21:00:00,9.1,33.2,0.3766,1022.7,790.0,951.6,912.9,903.4,1.4,5.8,221.3
7108,2010-12-31 22:00:00,9.6,34.6,0.4310,1044.4,767.3,861.9,889.2,1159.1,1.6,5.2,227.4
7109,2010-12-31 23:00:00,8.0,40.7,0.4085,952.8,691.9,908.5,917.0,1206.3,1.5,4.6,199.8


# Dataset Overview
Unfortunately, there are no official information about features. We can only guess their meaning:
<ul>
    <li>date_time - is date and time when sensors recording occured within interval of 1 hour</li>
    <li>deg_C - is a temperature measured in Celsius</li>
    <li>relative_himidity - is the measure of water vapor in the air, regardless of temperature</li>
    <li>absolute_humidity - also measures water vaport but <b>RELATIVE</b> to the temperature of the air</li>
</ul>

We also have three columns in the database whose values we want to predict using machine learning. What do they mean?
<ul>
    <li>target_carbon_monoxide - is a colorless, odorless, tasteless, flammable gas that is slightly less dense than air. Thermal combustion is the most common source of carbon monoxide.</li>
    <li>target_benzene - is an organic chemical compound. Because it contains only carbon and hydrogen atoms, benzene is classed as a hydrocarbon.</li>
    <li>target_nitrogen_oxides - in atmospheric chemistry, <b>NO<sub>x</sub></b> is a generic term for the nitrogen oxides that are most relevant for air pollution, namely nitric oxide (NO) and nitrogen dioxide (NO<sub>2</sub>). These gases contribute to the formation of smog and acid rain.</li>
</ul>

## Dataset description

In [34]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7111 entries, 0 to 7110
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   date_time               7111 non-null   object 
 1   deg_C                   7111 non-null   float64
 2   relative_humidity       7111 non-null   float64
 3   absolute_humidity       7111 non-null   float64
 4   sensor_1                7111 non-null   float64
 5   sensor_2                7111 non-null   float64
 6   sensor_3                7111 non-null   float64
 7   sensor_4                7111 non-null   float64
 8   sensor_5                7111 non-null   float64
 9   target_carbon_monoxide  7111 non-null   float64
 10  target_benzene          7111 non-null   float64
 11  target_nitrogen_oxides  7111 non-null   float64
dtypes: float64(11), object(1)
memory usage: 666.8+ KB


In [35]:
data_train.describe()

,deg_C,relative_humidity,absolute_humidity,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,target_carbon_monoxide,target_benzene,target_nitrogen_oxides
count,7111.000000,7111.000000,7111.000000,7111.000000,7111.000000,7111.000000,7111.000000,7111.000000,7111.000000,7111.000000,7111.000000
mean,20.878034,47.561004,1.110309,1091.572100,938.064970,883.903305,1513.238349,998.335565,2.086219,10.237083,204.066784
std,7.937917,17.398731,0.398950,218.537554,281.978988,310.456355,350.180310,381.537695,1.447109,7.694426,193.927723
min,1.300000,8.900000,0.198800,620.300000,364.000000,310.600000,552.900000,242.700000,0.100000,0.100000,1.900000
25%,14.900000,33.700000,0.855900,930.250000,734.900000,681.050000,1320.350000,722.850000,1.000000,4.500000,76.450000
50%,20.700000,47.300000,1.083500,1060.500000,914.200000,827.800000,1513.100000,928.700000,1.700000,8.500000,141.000000
75%,25.800000,60.800000,1.404150,1215.800000,1124.100000,1008.850000,1720.400000,1224.700000,2.800000,14.200000,260.000000
max,46.100000,90.800000,2.231000,2088.300000,2302.600000,2567.400000,2913.800000,2594.600000,12.500000,63.700000,1472.300000


## Null values

In [36]:
data_train.isnull().sum()

date_time                 0
deg_C                     0
relative_humidity         0
absolute_humidity         0
sensor_1                  0
sensor_2                  0
sensor_3                  0
sensor_4                  0
sensor_5                  0
target_carbon_monoxide    0
target_benzene            0
target_nitrogen_oxides    0
dtype: int64

# Data cleaning
First we will look for outliers and delete them if necessery

In [37]:
data_train

,date_time,deg_C,relative_humidity,absolute_humidity,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,target_carbon_monoxide,target_benzene,target_nitrogen_oxides
0,2010-03-10 18:00:00,13.1,46.0,0.7578,1387.2,1087.8,1056.0,1742.8,1293.4,2.5,12.0,167.7
1,2010-03-10 19:00:00,13.2,45.3,0.7255,1279.1,888.2,1197.5,1449.9,1010.9,2.1,9.9,98.9
2,2010-03-10 20:00:00,12.6,56.2,0.7502,1331.9,929.6,1060.2,1586.1,1117.0,2.2,9.2,127.1
3,2010-03-10 21:00:00,11.0,62.4,0.7867,1321.0,929.0,1102.9,1536.5,1263.2,2.2,9.7,177.2
4,2010-03-10 22:00:00,11.9,59.0,0.7888,1272.0,852.7,1180.9,1415.5,1132.2,1.5,6.4,121.8
...,...,...,...,...,...,...,...,...,...,...,...,...
7106,2010-12-31 20:00:00,9.2,32.0,0.3871,1000.5,811.2,873.0,909.0,910.5,1.3,5.1,191.1
7107,2010-12-31 21:00:00,9.1,33.2,0.3766,1022.7,790.0,951.6,912.9,903.4,1.4,5.8,221.3
7108,2010-12-31 22:00:00,9.6,34.6,0.4310,1044.4,767.3,861.9,889.2,1159.1,1.6,5.2,227.4
7109,2010-12-31 23:00:00,8.0,40.7,0.4085,952.8,691.9,908.5,917.0,1206.3,1.5,4.6,199.8


In [53]:
fig = make_subplots(rows=1, cols=2)

fig.add_trace(go.Box(
    y = data_train['deg_C'],
    name = 'Distribution',
    marker_color = '#990099'
), row=1, col=1)

fig.add_trace(go.Histogram(
    x = data_train['deg_C'],
), row=1, col=2)

# fig.add_trace(go.Scatter(
#     x = data_train.index,
#     y = data_train['deg_C']
# ), row=1, col=2)

fig.update_layout(
    template = 'plotly_dark',
    width = 1000
)

In [9]:
fig = go.Figure()

fig.add_trace(go.Box(
    y = data_train['deg_C'],
    name = 'Distribution',
    marker_color = '#990099'
))

fig.update_layout(
    template = 'plotly_dark',
    title_text = 'Box plot deg_C'
)

In [10]:
fig = go.Figure()

fig.add_trace(go.Box(
    y = data_train['relative_humidity'],
    name = 'Distribution',
    marker_color = '#b30000'
))

fig.update_layout(
    template = 'plotly_dark',
    title_text = 'Box plot relative_humidity'
)

In [11]:
fig = go.Figure()

fig.add_trace(go.Box(
    y = data_train['absolute_humidity'],
    name = 'Distribution',
    marker_color = '#b3b300'
))

fig.update_layout(
    template = 'plotly_dark',
    title_text = 'Box plot absolute_humidity'
)

In [12]:
fig = go.Figure()

fig.add_trace(go.Box(
    y = data_train['sensor_1'],
    name = 'Sensor 1'
))

fig.add_trace(go.Box(
    y = data_train['sensor_2'],
    name = 'Sensor 2'
))

fig.add_trace(go.Box(
    y = data_train['sensor_3'],
    name = 'Sensor 3'
))

fig.add_trace(go.Box(
    y = data_train['sensor_4'],
    name = 'Sensor 4'
))

fig.add_trace(go.Box(
    y = data_train['sensor_5'],
    name = 'Sensor 5'
))

fig.update_layout(
    template = 'plotly_dark',
    title_text = 'Box plot sensors'
)

What do the sensors represent?
I'm not really sure but after brief researche and reading notebooks I found something like this.
<ul>
    <li>Sensor_1 - (tin oxide) hourly averaged sensor response (nominally CO targeted)</li>
    <li>Sensor_2 - (titania) hourly averaged sensor response (nominally NMHC targeted)</li>
    <li>Sensor_3 - (tungsten oxide) hourly averaged sensor response (nominally NOx targeted)</li>
    <li>Sensor_4 - (tungsten oxide) hourly averaged sensor response (nominally NO2 targeted)</li>
    <li>Sensor_5 - (indium oxide) hourly averaged sensor response (nominally O3 targeted)</li>
</ul>

At first glance we see that sensors data might have some outliers.

# Data visualization 

In [18]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x = data_train['date_time'],
    y = data_train['deg_C'],
    marker_color = '#3366ff'
))

fig.update_layout(
    template = 'plotly_dark',
    width = 1000,
    title_text = 'Temperature over time'
)

In [22]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x = data_train['date_time'],
    y = data_train['relative_humidity'],
    marker_color = '#e6ac00'
))

fig.update_layout(
    template = 'plotly_dark',
    width = 1000,
    title_text = 'Relative humidity over time'
)

In [24]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x = data_train['date_time'],
    y = data_train['absolute_humidity'],
    marker_color = '#ff7800'
))

fig.update_layout(
    template = 'plotly_dark',
    width = 1000,
    title_text = 'Relative humidity over time'
)